In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [3]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)
    return str(year)+ '-' +str(month)+ '-' +str(day)

### 주요국가_중앙은행_금리

In [5]:
# file_name = '/finance/cetral_bank_interest_rate/cental_interest_' +cal_std_day(1)+ '.json'
# ctr_inter = spark.read.json(file_name, multiLine=True)
# ctr_inter.printSchema()

NameError: name 'sc' is not defined

In [11]:
data = []
    # 파이썬 압축해제 키워드
    # **kwargs => 여러 쌍의 키워드 args 가 넘어오면 받아서 dict로 반환
    # fnc(**dict) => dict에 있는 key-value 값들이 여러쌍의 kwargs 형태로 함수에 전달
for r1 in ctr_inter.select(ctr_inter.data, ctr_inter.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        temp = r2.asDict() # 액션매서드 row객체를 dict로 반환해주는 함수 asDIct()
#         print("출력-->", temp , '\n')
        temp['std_day'] = r1['meta.std_day'] # meta데이터에있는 날짜값 std_day는 키와 벨류를 지정해서 dict에 포함시켜줌
        data.append(Row(**temp))# Row에 **temp는 dict()을 kwargs로 전달해주는방법

ctr_inter_data = spark.createDataFrame(data)
ctr_inter_data.show(3)

+------------------+---------+-------+------------------+------------------+---------------+----------+
|          ctr_bank|cur_inter|is_rise|latest_chage_point|latest_change_date|next_conference|   std_day|
+------------------+---------+-------+------------------+------------------+---------------+----------+
|   한국 은행 (BOK)|    2.50%|   true|                25|        2022-08-25|     2022-10-12|2022-09-28|
|연방준비은행 (FED)|    3.25%|   true|                75|        2022-09-21|     2022-11-02|2022-09-28|
|유럽중앙은행 (ECB)|    1.25%|   true|                75|        2022-09-08|     2022-10-27|2022-09-28|
+------------------+---------+-------+------------------+------------------+---------------+----------+
only showing top 3 rows



## 주요국가 환율 

In [12]:
file_name = '/finance/exchange/sale_standard_rate_' +cal_std_day(0)+ '.json'
excng_rate = spark.read.json(file_name, multiLine=True)
excng_rate.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- country_name: string (nullable = true)
 |    |    |-- monetary_unit: string (nullable = true)
 |    |    |-- sale_standard_rate: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- cols: struct (nullable = true)
 |    |    |-- country_name: string (nullable = true)
 |    |    |-- monetary_unit: string (nullable = true)
 |    |    |-- sale_standard_rate: string (nullable = true)
 |    |-- desc: string (nullable = true)
 |    |-- std_day: string (nullable = true)



In [13]:
data = []
for r1 in excng_rate.select(excng_rate.data, excng_rate.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        temp = r2.asDict()
        temp['std_day'] = r1['meta.std_day'] 
        data.append(Row(**temp))
excng_rate_data = spark.createDataFrame(data)
excng_rate_data.show(3)

+------------+-------------+------------------+----------+
|country_name|monetary_unit|sale_standard_rate|   std_day|
+------------+-------------+------------------+----------+
|    대한민국|          KRW|                 1|2022-09-28|
|        미국|          USD|            1439.4|2022-09-28|
|    유럽연합|          EUR|            1375.2|2022-09-28|
+------------+-------------+------------------+----------+
only showing top 3 rows



## 유가 및 귀금속 

In [16]:
# 오일
file_name = '/finance/oil_gold/oil_price_' +cal_std_day(0)+ '.json'
oil_price = spark.read.json(file_name, multiLine=True)
oil_price.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- fluctuation_rate: string (nullable = true)
 |    |    |-- is_rise: boolean (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- unit: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- cols: struct (nullable = true)
 |    |    |-- fluctuation_rate: string (nullable = true)
 |    |    |-- is_rise: string (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- unit: string (nullable = true)
 |    |-- desc: string (nullable = true)
 |    |-- std_day: string (nullable = true)



In [17]:
# 오일
data = []
for r1 in oil_price.select(oil_price.data, oil_price.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        temp = r2.asDict()
        temp['std_day'] = r1['meta.std_day'] 
        data.append(Row(**temp))
oil_price_data = spark.createDataFrame(data)
oil_price_data.show(3)

+----------------+-------+-------+------------+---------+----------+
|fluctuation_rate|is_rise|  price|product_name|     unit|   std_day|
+----------------+-------+-------+------------+---------+----------+
|           -0.31|  false|1700.07|      휘발유|원 / 리터|2022-09-28|
|           -0.29|  false|1987.06|  고급휘발유|원 / 리터|2022-09-28|
|           -0.21|  false|1833.19|        경유|원 / 리터|2022-09-28|
+----------------+-------+-------+------------+---------+----------+
only showing top 3 rows



In [18]:
# 귀금속
file_name = '/finance/oil_gold/gold_price_' +cal_std_day(0)+ '.json'
gold_price = spark.read.json(file_name, multiLine=True)
gold_price.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- fluctuation_rate: string (nullable = true)
 |    |    |-- is_rise: boolean (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- unit: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- cols: struct (nullable = true)
 |    |    |-- fluctuation_rate: string (nullable = true)
 |    |    |-- is_rise: string (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- unit: string (nullable = true)
 |    |-- desc: string (nullable = true)
 |    |-- std_day: string (nullable = true)



In [19]:
# 귀금속
data = []
for r1 in gold_price.select(gold_price.data, gold_price.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        temp = r2.asDict()
        temp['std_day'] = r1['meta.std_day'] 
        data.append(Row(**temp))
gold_price_data = spark.createDataFrame(data)
gold_price_data.show(3)

+----------------+-------+--------+------------+-----------------+----------+
|fluctuation_rate|is_rise|   price|product_name|             unit|   std_day|
+----------------+-------+--------+------------+-----------------+----------+
|            0.30|   true|75123.89|     국내 금|           원 / g|2022-09-28|
|            0.17|   true| 1636.20|     국제 금|달러 / 트로이온스|2022-09-28|
|           -0.61|  false|  839.70|        백금|달러 / 트로이온스|2022-09-28|
+----------------+-------+--------+------------+-----------------+----------+
only showing top 3 rows



## 원자재 

In [20]:
# 농산물
file_name = '/finance/raw_materials/agriculture_' +cal_std_day(0)+ '.json'
agriculture = spark.read.json(file_name, multiLine=True)
agriculture.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- current_month_contract: string (nullable = true)
 |    |    |-- fluctuation_rate: string (nullable = true)
 |    |    |-- futures_exchange_name: string (nullable = true)
 |    |    |-- is_rise: boolean (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- unit: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- cols: struct (nullable = true)
 |    |    |-- current_month_contract: string (nullable = true)
 |    |    |-- fluctuation_rate: string (nullable = true)
 |    |    |-- futures_exchange_name: string (nullable = true)
 |    |    |-- is_rise: string (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- unit: string (nullable = true)
 |    |-- desc: string (nullable = true)
 |    |-- std_day: string (null


[Stage 24:>                                                         (0 + 1) / 1]



In [25]:
# 농산물
data = []
for r1 in agriculture.select(agriculture.data, agriculture.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        temp = r2.asDict()
        temp['std_day'] = r1['meta.std_day'] 
        data.append(Row(**temp))
agriculture_data = spark.createDataFrame(data)
agriculture_data.show(3)

+----------------------+----------------+---------------------+-------+-------+------------+-----------+----------+
|current_month_contract|fluctuation_rate|futures_exchange_name|is_rise|  price|product_name|       unit|   std_day|
+----------------------+----------------+---------------------+-------+-------+------------+-----------+----------+
|                 22-12|           +0.19|                 CBOT|   true| 667.50|      옥수수|  센트/부셸|2022-09-28|
|                 23-03|           -0.17|                  ICE|  false|  17.59|        설탕|센트/파운드|2022-09-28|
|                 22-11|           -0.23|                 CBOT|  false|1408.00|        대두|  센트/부셸|2022-09-28|
+----------------------+----------------+---------------------+-------+-------+------------+-----------+----------+
only showing top 3 rows



In [23]:
# 비금속 광물
file_name = '/finance/raw_materials/non_metal_' +cal_std_day(0)+ '.json'
non_metal = spark.read.json(file_name, multiLine=True)
non_metal.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- current_month_contract: string (nullable = true)
 |    |    |-- fluctuation_rate: string (nullable = true)
 |    |    |-- futures_exchange_name: string (nullable = true)
 |    |    |-- is_rise: boolean (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- unit: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- cols: struct (nullable = true)
 |    |    |-- fluctuation_rate: string (nullable = true)
 |    |    |-- futures_exchange_name: string (nullable = true)
 |    |    |-- is_rise: string (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- unit: string (nullable = true)
 |    |-- desc: string (nullable = true)
 |    |-- std_day: string (nullable = true)



In [26]:
# 비금속 광물
data = []
for r1 in non_metal.select(non_metal.data, non_metal.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        temp = r2.asDict()
        temp['std_day'] = r1['meta.std_day'] 
        data.append(Row(**temp))
non_metal_data = spark.createDataFrame(data)
non_metal_data.show(3)

+----------------------+----------------+---------------------+-------+-------+------------+-------+----------+
|current_month_contract|fluctuation_rate|futures_exchange_name|is_rise|  price|product_name|   unit|   std_day|
+----------------------+----------------+---------------------+-------+-------+------------+-------+----------+
|                  null|           +0.35|                  LME|   true|7475.00|        구리|달러/톤|2022-09-28|
|               달러/톤|             LME|                false|   null|  -1.24|          납|1754.00|2022-09-28|
|               달러/톤|             LME|                false|   null|  -0.20|        아연|2938.00|2022-09-28|
+----------------------+----------------+---------------------+-------+-------+------------+-------+----------+
only showing top 3 rows



In [22]:
# 에너지
file_name = '/finance/raw_materials/energy_' +cal_std_day(0)+ '.json'
energy = spark.read.json(file_name, multiLine=True)
energy.printSchema()


[Stage 26:>                                                         (0 + 1) / 1]



root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- current_month_contract: string (nullable = true)
 |    |    |-- fluctuation_rate: string (nullable = true)
 |    |    |-- futures_exchange_name: string (nullable = true)
 |    |    |-- is_rise: boolean (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- unit: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- cols: struct (nullable = true)
 |    |    |-- current_month_contract: string (nullable = true)
 |    |    |-- fluctuation_rate: string (nullable = true)
 |    |    |-- futures_exchange_name: string (nullable = true)
 |    |    |-- is_rise: string (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- unit: string (nullable = true)
 |    |-- desc: string (nullable = true)
 |    |-- std_day: string (null

In [27]:
# 비금속 광물
data = []
for r1 in energy.select(energy.data, energy.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        temp = r2.asDict()
        temp['std_day'] = r1['meta.std_day'] 
        data.append(Row(**temp))
energy_data = spark.createDataFrame(data)
energy_data.show(3)


[Stage 36:>                                                         (0 + 1) / 1]



+----------------------+----------------+---------------------+-------+------+------------+----------+----------+
|current_month_contract|fluctuation_rate|futures_exchange_name|is_rise| price|product_name|      unit|   std_day|
+----------------------+----------------+---------------------+-------+------+------------+----------+----------+
|                 22-11|           +2.84|                  ICE|   true|922.75|    가스오일|   달러/톤|2022-09-28|
|                 22-11|           +3.82|                NYMEX|   true|  3.16|      난방유| 달러/갤런|2022-09-28|
|                 22-11|           -3.62|                NYMEX|  false|  6.76|    천연가스|달러/MMBtu|2022-09-28|
+----------------------+----------------+---------------------+-------+------+------------+----------+----------+



In [34]:
# 주요국가 주가 지수
file_name = '/finance/stock_index/stock_index_' +cal_std_day(0)+ '.json'
stock_index = spark.read.json(file_name, multiLine=True)
stock_index.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- fluctuation_rate: string (nullable = true)
 |    |    |-- index_name: string (nullable = true)
 |    |    |-- is_rise: boolean (nullable = true)
 |    |    |-- price: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- cols: struct (nullable = true)
 |    |    |-- fluctuation_rate: string (nullable = true)
 |    |    |-- index_name: string (nullable = true)
 |    |    |-- is_rise: string (nullable = true)
 |    |    |-- price: string (nullable = true)
 |    |-- desc: string (nullable = true)
 |    |-- std_day: string (nullable = true)



In [35]:
# 주요국가 주가 지수
data = []
for r1 in stock_index.select(stock_index.data, stock_index.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        temp = r2.asDict()
        temp['std_day'] = r1['meta.std_day'] 
        data.append(Row(**temp))
stock_index_data = spark.createDataFrame(data)
stock_index_data.show(3)

+----------------+----------+-------+-------+----------+
|fluctuation_rate|index_name|is_rise|  price|   std_day|
+----------------+----------+-------+-------+----------+
|           +0.13|코스피지수|   true|2223.86|2022-09-28|
|           -0.08| 코스피 50|  false|1983.55|2022-09-28|
|           +0.83|    코스닥|   true| 698.11|2022-09-28|
+----------------+----------+-------+-------+----------+
only showing top 3 rows



## 주요국가 채권수익률

In [54]:
#  국채금리
file_name = '/finance/sovereign_yield/sovereign_yield_' +cal_std_day(0)+ '.json'
sovereign_yield = spark.read.json(file_name, multiLine=True)
sovereign_yield.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- 10y: string (nullable = true)
 |    |    |-- 1y: string (nullable = true)
 |    |    |-- 2y: string (nullable = true)
 |    |    |-- 30y: string (nullable = true)
 |    |    |-- 3y: string (nullable = true)
 |    |    |-- 5y: string (nullable = true)
 |    |    |-- country_name: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- cols: struct (nullable = true)
 |    |    |-- 10y: string (nullable = true)
 |    |    |-- 1y: string (nullable = true)
 |    |    |-- 2y: string (nullable = true)
 |    |    |-- 30y: string (nullable = true)
 |    |    |-- 3y: string (nullable = true)
 |    |    |-- 5y: string (nullable = true)
 |    |    |-- country_name: string (nullable = true)
 |    |-- desc: string (nullable = true)
 |    |-- std_day: string (nullable = true)




[Stage 40:>                                                         (0 + 1) / 1]



In [55]:
data = []
for r1 in sovereign_yield.select(sovereign_yield.data, sovereign_yield.meta.std_day).toLocalIterator():
    for r2 in r1.data:
        temp = r2.asDict()
        temp['std_day'] = r1['meta.std_day'] 
        data.append(Row(**temp))
sovereign_yield_data = spark.createDataFrame(data)
sovereign_yield_data.show(10)

+------+------+------+-----+------+------+------------+----------+
|   10y|    1y|    2y|  30y|    3y|    5y|country_name|   std_day|
+------+------+------+-----+------+------+------------+----------+
| 4.231| 3.440| 4.266|3.983| 4.216| 4.323|        한국|2022-09-29|
| 3.963|4.1460|4.3080|3.828|4.4000|4.2140|        미국|2022-09-29|
|4.0070|3.8730|4.2440|3.934|4.3120|4.2720|        영국|2022-09-29|
| 0.250|-0.097|-0.036|1.471|-0.025| 0.083|        일본|2022-09-29|
+------+------+------+-----+------+------+------------+----------+



In [56]:
pd_sv = sovereign_yield_data.to_pandas_on_spark()


In [57]:
pd_sv = pd_sv.set_index(['country_name','std_day'])
pd_sv = pd_sv.stack()
pd_sv= pd_sv.to_dataframe('bond_yield')


In [58]:
sovereign_yield_data= pd_sv.reset_index().rename(columns={'level_2':'y_cnt'}).to_spark()
sovereign_yield_data = sovereign_yield_data.select('*',(col('bond_yield').contains('-')).alias('is_rise'))

In [59]:
sovereign_yield_data.show(3)

+------------+----------+-----+----------+-------+
|country_name|   std_day|y_cnt|bond_yield|is_rise|
+------------+----------+-----+----------+-------+
|        한국|2022-09-29|   3y|     4.216|  false|
|        한국|2022-09-29|   5y|     4.323|  false|
|        한국|2022-09-29|  10y|     4.231|  false|
+------------+----------+-----+----------+-------+
only showing top 3 rows

